In [ ]:
import torch
import torch.nn as nn
from torchview import draw_graph
import mvit
import graphviz
graphviz.set_jupyter_format('png')
model = mvit.mobilevit_xxs()
graph = draw_graph(model, (torch.randn(1, 3, 320, 320), ),device='cpu')
graph.visual_graph

In [5]:
from torchviz import make_dot
y = model(torch.randn(1, 3, 320, 320))
out = 0
for i in y:
    out += i.mean()
make_dot(out, params=dict(model.named_parameters()))

In [ ]:
    print("=" * 10, "test torch_pruning", "=" * 10)
    imp = tp.importance.TaylorImportance()
    iterative_steps = 5
    ignore_layers = []
    for i in model.modules():
        if isinstance(i, mvit.Attention):
            ignore_layers.append(i)
    pruner = tp.pruner.MagnitudePruner(
        model,
        example_input,
        importance=imp,
        iterative_steps=iterative_steps,
        ch_sparsity=0.7,
        ignored_layers=ignore_layers,
    )
    base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_input)
    print(f"base macs: {base_macs}, base params: {base_nparams}")
    for i in range(iterative_steps):
        if isinstance(imp, tp.importance.TaylorImportance):
            # Taylor expansion requires gradients for importance estimation
            y = model(example_input)  # a dummy loss for TaylorImportance
            loss = 0
            for i in y:
                loss += i.mean()
            loss.backward()  # before pruner.step()
        pruner.step()
        macs, nparams = tp.utils.count_ops_and_params(model, example_input)
        print(f"step {i}: macs: {macs}, params: {nparams}")
    print("=" * 10, "test torch_pruning finished", "=" * 10)